In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('./drive/MyDrive/sharif/Spider/ipython(guide)')

In [7]:
import re

In [4]:
import pandas as pd
data = pd.read_csv('../Data/unpreprocessed_dataset.csv')

In [19]:
count_data = data.copy()
count_data = count_data.dropna().reset_index(drop=True)
count = {}
tags_list = pd.read_excel('../Tags.xlsx',header = None)
tags_list.columns = ['tag']
for _,tag in tags_list.iterrows():
  if tag['tag']!='ویرگول' and tag['tag']!=None and tag['tag']!='':
    count[re.sub(' ','',tag['tag'].lower())] = 0

for indx,d in count_data.iterrows():
  tags = re.sub('\'|\[|\]|ویرگول','',d['tag']).split(',')
  
  for tag in tags:
    if tag.strip().lower() in tags_list.values:
      count[tag.strip().lower().replace(' ','')]+=1 


In [28]:
l = sorted(list(count.keys()))
for i in range(len(l)):
  print(l[i],' : ',count[l[i]])

api  :  581
c  :  3735
css  :  1284
designpattern  :  88
devops  :  68
golang  :  80
html  :  1410
ios  :  261
nodejs  :  222
oop  :  585
php  :  1008
reactnative  :  34
redux  :  57
ubuntu  :  84
vscode  :  81
آموزشبرنامهنویسی  :  1264
آموزشپایتون  :  519
استارتاپ  :  2839
استخدام  :  631
الگوریتم  :  311
امنیت  :  772
اندروید  :  722
اندرویداستودیو  :  203
اپلیکیشن  :  738
اینترنت  :  370
اینستاگرام  :  1020
برنامهنویسی  :  949
برنامهنویسیاندروید  :  808
برنامهنویسیوب  :  636
تجربهکاربری  :  784
تحلیلداده  :  283
تلگرام  :  482
توسعهدهنده  :  551
توسعهوب  :  1099
تکنولوژی  :  509
جاوا  :  920
جاوااسکریپت  :  1136
جنگو  :  69
دادهکاوی  :  222
داکر  :  160
دیجیتالمارکتینگ  :  1176
رابطکاربری  :  576
ریاکت  :  405
زبانبرنامهنویسی  :  1040
سئو  :  1167
سیشارپ  :  119
شیگرایی  :  175
طراحیسایت  :  1340
طراحیوب  :  1482
علمداده  :  289
فرانتاند  :  155
فلاتر  :  121
فناوری  :  286
لاراول  :  466
لینوکس  :  763
مصاحبه  :  211
مهارت  :  282
مهندسینرمافزار  :  201
موفقیت  :  3061
نرمافزار  : 

### Libs

In [ ]:
! pip install scitime
! pip install scikit-multilearn

     |████████████████████████████████| 45.9MB 93kB/s 
     |████████████████████████████████| 92kB 3.8MB/s 


In [ ]:
from sklearn.model_selection import train_test_split , cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import accuracy_score,roc_curve
from sklearn.metrics import plot_confusion_matrix,precision_score,recall_score,\
        classification_report,f1_score,confusion_matrix
import matplotlib.pyplot as plt
import numpy
import time
import pandas
from joblib import dump
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
import scitime
import re
import ast


## 1 - Collecting Data


In [ ]:
X  = pandas.read_csv('../Data/features.csv')
y = pandas.read_csv('../Data/preprocessed.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += re.sub('\"|\'| |=','',item.lower()).split(",")
  tag.append(re.sub('\"|\'| |=','',item.lower()).split(","))
labels = list(set(labels))
Y=MultiLabelBinarizer().fit_transform(tag)

In [ ]:
print(X.shape)
print(Y.shape)
print(len(labels))
class_num=len(labels)

(35667, 486)
(35667, 80)
80


## 2 - Spliting Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape , )

train:  (21399, 486) 
test:  (7134, 486) 
val:  (7134, 486)


## 3 - EDA

## 4 - Model Selection

SVC

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
clf_SVC_basic = svm.SVC(kernel="rbf")
clf_SVC_OneVsRest = OneVsRestClassifier(estimator=clf_SVC_basic)

In [ ]:
clf_SVC = clf_SVC_OneVsRest.fit(X_train,y_train)

In [ ]:
clf_SVC = load('clf_SVC.joblib') # Loading trained model

In [ ]:
y_pred_SVC = clf_SVC.predict(X_test)

In [ ]:
dump(clf_SVC, 'clf_SVC.joblib') 

['clf_SVC.joblib']

In [ ]:
print(accuracy_score(y_test, y_pred_SVC))
print(classification_report(y_test, y_pred_SVC))

0.2636669470142977

Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
from skmultilearn.problem_transform import BinaryRelevance
clf_naive_bayes = BinaryRelevance(GaussianNB())
clf_nb = clf_naive_bayes.fit(X_train,y_train)

In [ ]:
y_pred_nb = clf_nb.predict(X_test)

In [ ]:
dump(clf_nb, 'clf_nb.joblib') 

['clf_nb.joblib']

In [ ]:
print(accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

0.027894589290720494

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

forest = RandomForestClassifier(random_state=1)
clf_multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
clf_mtf = clf_multi_target_forest.fit(X_train, y_train)

In [ ]:
y_pred_mtf = clf_mtf.predict(X_test)

In [ ]:
dump(clf_SVC, 'clf_mtf.joblib') 

['clf_mtf.joblib']

In [ ]:
print(accuracy_score(y_test, y_pred_mtf))
print(classification_report(y_test, y_pred_mtf))

0.15068685169610316

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
dump(neigh, 'neigh_mtf.joblib') 

['neigh_mtf.joblib']

In [ ]:
y_pred_neigh=neigh.predict(X_test)
# print(y_pred)

In [ ]:
print(accuracy_score(y_test, y_pred_neigh))
print(classification_report(y_test, y_pred_neigh))

0.25203252032520324
              precision    recall  f1-score   support

           0       0.49      0.19      0.28       119
           1       0.57      0.45      0.50       732
           2       1.00      0.05      0.10        19
           3       0.51      0.25      0.33       258
           4       0.20      0.07      0.10        15
           5       0.50      0.08      0.13        13
           6       0.00      0.00      0.00        11
           7       0.38      0.18      0.24       286
           8       0.33      0.02      0.03        55
           9       0.58      0.14      0.23        49
          10       0.36      0.07      0.12       117
          11       0.55      0.16      0.25       222
          12       0.00      0.00      0.00         7
          13       0.33      0.18      0.24        11
          14       0.67      0.10      0.17        21
          15       0.33      0.05      0.08        21
          16       0.42      0.16      0.23       257
       

## 5 - Set Hyperparameters

## 6 - Test Model

In [ ]:
classification_rep=classification_report(y_test, y_pred)
print(classification_rep)
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred,average='macro')
recall_macro = recall_score(y_test, y_pred,average='macro')
f1score_macro=f1_score(y_test, y_pred,average='macro')

precision_weighted = precision_score(y_test, y_pred,average='weighted')
recall_weighted = recall_score(y_test, y_pred,average='weighted')
f1score_weighted =f1_score(y_test, y_pred,average='weighted')

#ROC 
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

with open("knn_evaluation.txt", "a") as file:
        file.write("Results:"+"\n" +
                   "classification report:\n"+classification_rep+"\n"
                   +"accuracy="+ str(accuracy)+"\n"
                   +"macro precision="+str(precision_macro)+"\n"
                   +"macro recall="+str(recall_macro)+"\n"
                   +"macro f-score="+str(f1score_macro)+"\n"
                   +"\n"
                   + "weighted precision=" + str(precision_weighted) + "\n"
                   + "weighted recall=" + str(recall_weighted) + "\n"
                   + "weighted f-score=" + str(f1score_weighted) + "\n"
                   )